In [2]:
import pandas as pd
import random
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm

# 데이터를 불러옵니다.
train = pd.read_csv('./train.csv')

# Word2Vec 모델을 불러옵니다.
w2v_path = '/aiffel/data/ko.bin'  
wv_model = Word2Vec.load(w2v_path)

def lexical_sub(sentence, word2vec):
    toks = sentence.split()
    
    for idx, _from in enumerate(toks):
        if (idx + 1) % 4 == 0:  # 토큰의 4의 배수마다
            try:
                _to = word2vec.wv.most_similar(_from)[0][0]  # 수정된 부분
                toks[idx] = _to
            except:   # 단어장에 없는 단어
                continue

    return ' '.join(toks)

# Mecab으로 토큰화
mecab = Mecab()
train['conversation_mecab'] = train['conversation'].apply(lambda x: " ".join(mecab.morphs(x)))

# 증강된 문장을 저장할 리스트
augmented_sentences = []

# 각 문장에 대해 증강
for sentence in tqdm(train['conversation_mecab']):
    augmented_sentence = lexical_sub(sentence, wv_model)
    augmented_sentences.append(augmented_sentence)

# 증강된 데이터를 새로운 데이터프레임에 저장
augmented_df = pd.DataFrame({
    'conversation': augmented_sentences,
    'class': train['class']
})

# 결과를 train_augmented_wv.csv로 저장
augmented_df.to_csv('./train_augmented_wv.csv', index=False)


100%|██████████| 3950/3950 [03:32<00:00, 18.55it/s]


In [7]:
!pip install gensim==3.8.1

     |████████████████████████████████| 23.4 MB 6.2 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.1-cp39-cp39-linux_x86_64.whl size=23985908 sha256=1c70609edaf0f7bf0f17e6eac8da138e87a5dbbad3c28079b7b9704aaefce63e
  Stored in directory: /aiffel/.cache/pip/wheels/fe/77/24/b3ffc86119703277a8aeb0e6914e97e216b1bd45287e41d6ba
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2
